In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/codebasics/deep-learning-keras-tf-tutorial/master/6_gradient_descent/insurance_data.csv")
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [3]:
from sklearn.model_selection import train_test_split

In [4]:
X_train,X_test,y_train,y_test = train_test_split(df[['age','affordibility']],df.bought_insurance,test_size=0.2,random_state=25)

In [5]:
print(len(df))

28


In [6]:
print(len(X_train))
print(len(X_test))

22
6


In [7]:
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age']/100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age']/100

In [8]:
X_train_scaled

,age,affordibility
0,0.22,1
13,0.29,0
6,0.55,0
17,0.58,1
24,0.50,1
19,0.18,1
25,0.54,1
16,0.25,0
20,0.21,1
3,0.52,0


> Keras model

In [9]:
model = keras.Sequential([
    keras.layers.Dense(1,input_shape=(2,),activation = 'sigmoid',kernel_initializer = 'ones', bias_initializer = 'zeros')
])

# neuron is 1
#input shape is 2 - age and affordibility
# output is 1 -whether person buying insurance or not

In [12]:
model.compile(optimizer='adam',
                loss = 'binary_crossentropy',
                metrics = ['accuracy'])
# binary cross entropy same as log loss

model.fit(X_train_scaled,y_train,epochs=2000)

Epoch 1/2000
1/1 [==============================] - 1s 992ms/step - loss: 0.5569 - accuracy: 0.6818
Epoch 2/2000
1/1 [==============================] - 0s 7ms/step - loss: 0.5569 - accuracy: 0.6818
Epoch 3/2000
1/1 [==============================] - 0s 6ms/step - loss: 0.5568 - accuracy: 0.6818
Epoch 4/2000
1/1 [==============================] - 0s 11ms/step - loss: 0.5568 - accuracy: 0.6818
Epoch 5/2000
1/1 [==============================] - 0s 8ms/step - loss: 0.5567 - accuracy: 0.6818
Epoch 6/2000
1/1 [==============================] - 0s 9ms/step - loss: 0.5566 - accuracy: 0.6818
Epoch 7/2000
1/1 [==============================] - 0s 11ms/step - loss: 0.5566 - accuracy: 0.6818
Epoch 8/2000
1/1 [==============================] - 0s 7ms/step - loss: 0.5565 - accuracy: 0.6818
Epoch 9/2000
1/1 [==============================] - 0s 7ms/step - loss: 0.5565 - accuracy: 0.6818
Epoch 10/2000
1/1 [==============================] - 0s 7ms/step - loss: 0.5564 - accuracy: 0.6818
Epoch 11/2000
1

In [13]:
model.evaluate(X_test_scaled,y_test)

1/1 [==============================] - 3s 3s/step - loss: 0.3993 - accuracy: 0.8333


[0.39934563636779785, 0.8333333134651184]

In [14]:
X_test_scaled

,age,affordibility
2,0.47,1
10,0.18,1
21,0.26,0
11,0.28,1
14,0.49,1
9,0.61,1


In [15]:
model.predict(X_test_scaled)

1/1 [==============================] - 0s 292ms/step


array([[0.69123197],
       [0.41387698],
       [0.18201381],
       [0.5124767 ],
       [0.70795256],
       [0.796233  ]], dtype=float32)

In [16]:
y_test

2     1
10    0
21    0
11    0
14    1
9     1
Name: bought_insurance, dtype: int64

In [18]:
coef,intercept = model.get_weights()
coef,intercept

(array([[3.9787805],
        [1.4731042]], dtype=float32),
 array([-2.5372458], dtype=float32))

> without tensor flowm

In [19]:
def sigmoid(x):
    import math
    return 1/(1+math.exp(-x))
sigmoid(18)

0.9999999847700205

In [20]:
def prediction_function(age,affordibility):
    weighted_sum = coef[0]*age + coef[1] * affordibility + intercept
    return sigmoid(weighted_sum)

In [22]:
print(prediction_function(.47,1))
print(prediction_function(.18,1))
print(prediction_function(.26,0))

0.691231997452683
0.41387697282726926
0.18201380654538601


In [31]:
def log_loss(y_true,y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(i,epsilon) for i in y_predicted]
    y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

In [25]:
def sigmoid_numpy(X):
    return 1/(1+ np.exp(-X))

sigmoid_numpy(np.array([12,0,1]))

array([0.99999386, 0.5       , 0.73105858])

In [35]:
def gradient_descent(age,affordability,y_true,epochs,loss_threshold):
    #w1,w2,bias
    w1 = w2 = 1
    bias =0
    rate = 0.5
    n=len(age)

    for i in range(epochs):
        weighted_sum = w1 * age +w2 * affordability +bias
        y_predicted = sigmoid_numpy(weighted_sum)
        loss = log_loss(y_true,y_predicted)
        w1d = (1/n)*np.dot(np.transpose(age),(y_predicted-y_true))
        w2d = (1/n)*np.dot(np.transpose(affordability),(y_predicted-y_true))
        bias_d = np.mean(y_predicted-y_true)

        w1 = w1 -rate * w1d
        w2 = w2 - rate * w2d
        bias = bias - rate* bias_d
        print(f'Epoch : {i},w1:{w1}, w2: {w2}, bias: {bias}, loss: {loss}')
        if loss<=loss_threshold:
            break
    return w1, w2, bias

In [36]:

gradient_descent(X_train_scaled['age'],X_train_scaled['affordibility'],y_train,1000,0.4860)

Epoch : 0,w1:0.974907633470177, w2: 0.948348125394529, bias: -0.11341867736368583, loss: 0.7113403233723417
Epoch : 1,w1:0.9556229728273669, w2: 0.9058873696677865, bias: -0.2122349122718517, loss: 0.6812647787377568
Epoch : 2,w1:0.9416488476693794, w2: 0.8719790823960313, bias: -0.29775789977965383, loss: 0.6591474252715025
Epoch : 3,w1:0.9323916996249162, w2: 0.8457541517722915, bias: -0.37150947240035115, loss: 0.6431523291301916
Epoch : 4,w1:0.9272267472726993, w2: 0.8262362885332687, bias: -0.4350664302689159, loss: 0.6316873063379158
Epoch : 5,w1:0.9255469396815343, w2: 0.8124402814952774, bias: -0.4899449005893882, loss: 0.6234717079975919
Epoch : 6,w1:0.9267936114129968, w2: 0.8034375029757677, bias: -0.5375299543522855, loss: 0.6175321183044205
Epoch : 7,w1:0.93047170420295, w2: 0.7983920007454487, bias: -0.5790424270894964, loss: 0.6131591858705934
Epoch : 8,w1:0.9361540784567943, w2: 0.7965748796787705, bias: -0.6155315088627656, loss: 0.6098518179750948
Epoch : 9,w1:0.94347

(4.013186684903757, 1.4248846178557852, -2.50777676442325)

In [37]:
coef,intercept

(array([[3.9787805],
        [1.4731042]], dtype=float32),
 array([-2.5372458], dtype=float32))

> using tensor flow -  w1 = 3.9787805 , w2 = 1.4731042, bias = -2.5372458<br>
> using python code -  w1 = 4.0131866 , w2 = 1.4248846, bias = -2.5077767


In [52]:
class MyNN:
    def __init__(self):
        self.w1 = 1
        self.w2 = 1
        self.bias = 0

    def fit(self, X,y,epochs,loss_threshold):
        self.w1,self.w2,self.bias = self.gradient_descent(X['age'],X['affordibility'],y, epochs, loss_threshold)
    def predict(self, X_test):
        weighted_sum = self.w1 * X_test['age'] + self.w2 * X_test['affordibility']+self.bias
        
    def gradient_descent(self, age,affordability,y_true,epochs,loss_threshold):
    #w1,w2,bias
        w1 = w2 = 1
        bias =0
        rate = 0.5
        n=len(age)

        for i in range(epochs):
            weighted_sum = w1 * age +w2 * affordability +bias
            y_predicted = sigmoid_numpy(weighted_sum)
            loss = log_loss(y_true,y_predicted)
            w1d = (1/n)*np.dot(np.transpose(age),(y_predicted-y_true))
            w2d = (1/n)*np.dot(np.transpose(affordability),(y_predicted-y_true))
            bias_d = np.mean(y_predicted-y_true)

            w1 = w1 -rate * w1d
            w2 = w2 - rate * w2d
            bias = bias - rate* bias_d
            if i %50==0:
                print(f'Epoch : {i},w1:{w1}, w2: {w2}, bias: {bias}, loss: {loss}')
            if loss<=loss_threshold:
                break
        return w1, w2, bias

In [54]:
customModel = MyNN()
customModel.fit(X_train_scaled,y_train,epochs = 100, loss_threshold = 0.4860)

Epoch : 0,w1:0.974907633470177, w2: 0.948348125394529, bias: -0.11341867736368583, loss: 0.7113403233723417
Epoch : 50,w1:1.503319554173139, w2: 1.108384790367645, bias: -1.2319047301235464, loss: 0.5675865113475955


In [51]:
customModel.predict(X_test_scaled)

AttributeError: 'MyNN' object has no attribute 'predict'